In [2]:
## data collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

##load dataset
data=gutenberg.raw('shakespeare-hamlet.txt')

##save file
with open('hamlet.txt','w')as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\MUSKAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


In [ ]:
##data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

##load dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()

##tokenize
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])      #Splits it into words (tokens),Builds a vocabulary of unique words,Assigns each word a unique integer index.
total_words=len(tokenizer.word_index)+1
total_words

4818

In [10]:
##create input sequences
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]   #Example: "The cat sat" → [2, 5, 9]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)      
'''This generates all possible prefix n-gram sequences from the line: For [2, 5, 9], it produces:[2, 5] [2, 5, 9]
 These are sequences where the model could learn: "Given [2, 5], predict the next word'''
        

'This generates all possible prefix n-gram sequences from the line: For [2, 5, 9], it produces:[2, 5] [2, 5, 9]\n These are sequences where the model could learn: "Given [2, 5], predict the next word'

In [12]:
##pad sequence
max_sequence_len=max([len(x)for x in input_sequences])
max_sequence_len

14

In [13]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [14]:
###create predictor and label
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words) ##onehot encoded while multi class

In [16]:
#split into train n test split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [17]:
#define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [ ]:
##train LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU

##define model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))  #Randomly "drops" 20% of the outputs from the previous LSTM layer during training. Prevents overfitting by ensuring the model doesn’t rely too much on any one node.
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

##compile model
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_2 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_1 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               100400    
                                                                 
 dense_1 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
##train model
history=model.fit(x_train,y_train,epochs=30,validation_data=(x_test,y_test),verbose=1,callbacks=[early_stopping])

Epoch 1/30
644/644 [==============================] - 17s 26ms/step - loss: 6.2350 - accuracy: 0.0479 - val_loss: 7.0757 - val_accuracy: 0.0433
Epoch 2/30
644/644 [==============================] - 16s 25ms/step - loss: 6.0855 - accuracy: 0.0530 - val_loss: 7.1748 - val_accuracy: 0.0449
Epoch 3/30
644/644 [==============================] - 18s 28ms/step - loss: 5.9622 - accuracy: 0.0545 - val_loss: 7.1957 - val_accuracy: 0.0517
Epoch 4/30
644/644 [==============================] - 17s 26ms/step - loss: 5.8431 - accuracy: 0.0603 - val_loss: 7.2601 - val_accuracy: 0.0552


In [32]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    # Convert the input text into a sequence of tokens (integers)
    token_list = tokenizer.texts_to_sequences(text)[0]
    
    # If the sequence is longer than the max_sequence_len, truncate it
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    
    # Pad the sequence to ensure it's the right length for the model
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    
    # Get the model's prediction for the next word (probabilities for each word)
    predicted = model.predict(token_list, verbose=1)
    
    # Find the index of the word with the highest predicted probability
    predicted_word_index = np.argmax(predicted, axis=1)
    
    # Look up the word corresponding to the predicted index
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word  # Return the predicted word
    
    # If no word is found, return None
    return None


In [33]:
input_text="Barn. Last night of all,When yond same the"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:Barn. Last night of all,When yond same the
1/1 [==============================] - 0s 31ms/step
Next Word PRediction:the


In [34]:
## Save the model
model.save("next_word_lstm.h5")
## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

c:\Users\MUSKAN\data_science_muskan\RNN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
